In [1]:
import os

In [111]:
folder = '/Users/akprakash/Programming/modelica/modelica-json/java/parser/src/main/java/gov/lbl/parser/domain'
op_folder = '/Users/akprakash/Programming/modelica/modelica-json/jsParser/domain/'

In [134]:
def convert(file, folder):
    js=None
    with open(os.path.join(folder, file)) as fp:
        contents = fp.readlines()
        
        class_name = None
        i = 0
        js = ""
        while i < len(contents):
            l = contents[i]
            line = l.strip().split('\n')[0]
            if (line == '\n' or line == ''):
                i+=1
                continue
                
            if line.startswith("public class "):
                class_name = line.split("public class ")[1].split(' ')[0]
                js+="class {} {{\n".format(class_name)
            elif class_name is not None:
                if line.startswith("public {}".format(class_name)):
                    if "(" in line and ")" in line:
                        variables = line.split("(")[1].split(")")[0].split(",")
                        constructor_line = "\tconstructor("
                        variable_expr = ""
                        for variable in variables:
                            variable_name = variable.strip().split(' ')[1]
                            constructor_line+=variable_name+", "
                            variable_expr += "\t\tthis.{} = {};\n".format(variable_name, variable_name)
                        constructor_line=constructor_line[0: len(constructor_line)-2]+") {\n"
                        js+=constructor_line
                        js+=variable_expr
                        js+="\t}\n"
                elif line.startswith("this."):
                    i+=1
                    continue
                elif line.startswith("private"):
                    i+=1
                    continue
            i+=1
        js+="}\n"
        js+="exports.{} = {}\n".format(class_name, class_name)
    
    return js
            
            

In [136]:
# class class_name {
#   constructor(method1, method2) {
#     this.method1 = method1;
#     this.method2= method2;
#   }
# }

In [137]:
java_files = os.listdir(folder)
print(len(java_files))
for java_file in java_files:
    if java_file.endswith('.java'):
        op = convert(java_file, folder)
        if op is None:
            print(java_file)
        else:
            with open("{}.js".format(op_folder+java_file.split('.java')[0]), "w") as fp:
                fp.write(op)
js_files = os.listdir(op_folder)
print(len(js_files))

91
91
